### Photonic lantern empirical identification procedure: pure linear algebra derivation

In [1]:
import numpy as np

In [2]:
N = 19
B = np.random.uniform(low=-1, high=1, size=(N,N)) + 1j * np.random.uniform(low=-1, high=1, size=(N,N))

We're never allowed to look at B; instead we can only look at the outputs of this function:

In [3]:
def query(x):
    return np.abs(np.dot(B, x)) ** 2

In [4]:
absB = np.zeros((N,N))
diagonal_query = np.zeros(N)
for i in range(N):
    diagonal_query[:] = 0.0
    diagonal_query[i] = 1.0
    absB[:,i] = np.sqrt(query(diagonal_query))

In [5]:
np.allclose(np.abs(B), absB)

True

In [12]:
def cross_term_coefficients(absA, query_coeff):
    coeffs = []
    for i in range(N):
        for j in range(i):
            amplitude_factor = 2 * absA[i] * abs(query_coeff[i]) * absA[j] * abs(query_coeff[j])
            phase_query = np.angle(query_coeff[i]) - np.angle(query_coeff[j])
            coeffs.append(amplitude_factor * np.cos(phase_query)) # coefficient on cos(k_matrix[i] - k_matrix[j]
            coeffs.append(-amplitude_factor * np.sin(phase_query)) # coefficient on sin(k_matrix[i] - k_matrix[j]
            
    return coeffs

def remove_diagonal_terms_from_query(x):
    N = len(x)
    res = query(x)
    for i in range(N):
        y = np.zeros(N, dtype=np.complex128)
        y[i] = x[i]
        res -= query(y)
        
    return res

def find_phase_from_sine_cos(sinx, cosx):
    return np.sign(sinx) * np.arccos(cosx)

def find_phase_diffs(absA):
    N = absA.shape[0]
    coeffs_set = np.zeros((N, N*(N-1), N*(N-1)), dtype=np.complex128)
    results_set = np.zeros((N, N*(N-1)))
    
    # one system per port, each of which has N*(N-1) queries to find N*(N-1) sin/cos differences
    for i in range(N*(N-1)):
        x = np.random.uniform(-1, 1, N) + 1j * np.random.uniform(-1, 1, N)
        res_cross = remove_diagonal_terms_from_query(x)
        for port_idx in range(N):
            coeffs_set[port_idx,i,:] = np.array(cross_term_coefficients(absA[port_idx,:], x))
            results_set[port_idx,i] = res_cross[port_idx]
            
    sines_and_cosines = [np.real(np.linalg.solve(c, r)) for (c, r) in zip(coeffs_set, results_set)]
    phase_diffs_matrix = np.zeros((N,N*(N-1)//2))
    for (i, row) in enumerate(sines_and_cosines):
        for (j, (s, c)) in enumerate(zip(row[::2], row[1::2])):
            phase_diffs_matrix[i,j] = find_phase_from_sine_cos(c, s)
            
    return phase_diffs_matrix

def phases_from_diffs(diffs):
    phases = np.zeros((N,N))
    lv = 0
    for i in range(1, N):
        phases[:,i] = diffs[:,lv]
        lv += i
            
    return phases

In [14]:
phase_diffs = find_phase_diffs(absB)
phases_recon = phases_from_diffs(phase_diffs)
B_recon = absB * np.exp(1j * phases_recon)

In [15]:
for _ in range(100):
    x = np.random.uniform(low=-1, high=1, size=N) + 1j * np.random.uniform(low=-1, high=1, size=N)
    assert np.allclose(query(x), np.abs(B_recon @ x) ** 2)